## Planning to build this Architecture

![My First Board (1).jpg](<attachment:My First Board (1).jpg>)

In [1]:
import os
import sys
from pathlib import Path

current_dir = Path().cwd()
project_root = current_dir.parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print("done")

done


In [2]:
import logfire
import pandas as pd
import numpy as np
from pydantic_ai import Agent, RunContext
from pydantic_ai.providers.ollama import OllamaProvider
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.models.google import GoogleModel
from pydantic_ai.models.groq import GroqModel
from pydantic_ai.providers.groq import GroqProvider
from pydantic_ai.providers.google import GoogleProvider
from src.utils.save_data import load_df_parquet_safe
from src.utils.datamodels import Blink

logfire.configure()

Logfire project URL: ]8;id=791206;https://logfire-eu.pydantic.dev/kkadi8434367970/mmr\https://logfire-eu.pydantic.dev/kkadi8434367970/mmr]8;;\

In [3]:
logfire.info("hello, {place}!", place='World')

21:11:33.236 hello, World!


In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
api_key = os.getenv("GEMINI_API_KEY")

In [6]:
path = project_root / "data" / "processed" / "merge" / "final_dataframe.parquet"
df = load_df_parquet_safe(path=str(path))

### Gemini

In [7]:
provider = GoogleProvider(api_key=api_key)
model = GoogleModel('gemini-2.5-flash-lite', provider=provider)
agent = Agent(model)

In [8]:
result = await agent.run(user_prompt="Hey! Tomorrow is going to be fun")
print(result.output)

That's awesome to hear! What makes you say tomorrow is going to be fun? I'm curious to know what you have planned or what you're looking forward to! 😊


### Local Ollama model

In [ ]:
model = OpenAIChatModel(
    model_name='llama3.2',
    provider=OllamaProvider(
        base_url='http://localhost:11434/v1'
    ),
)

In [44]:
agent = Agent(
    model,
    deps_type=Blink
    )
# result = await agent.run(user_prompt="hii! there")
# print(result.output)

In [15]:
df['blinking_data'].iloc[0]

{'intensity': None,
 'asymmetry': None,
 'is_blinking': True,
 'rz_score': 3.57146416776886,
 'is_anomalous': False,
 'continuos_anomaly': False,
 'part_of_anomalous_range': None}

In [40]:
for i in range(len(df['blinking_data'])):
    if df['blinking_data'].iloc[i]['is_anomalous'] == True:
        print(i)

49
69
78
79
80
81
135
141
179
212
244
277
292
295
296
298
322
330
331
332
333
342
373
461
473
481
575
586
588
590
602
603
604
620
622
624
815
816
817
818
819
820
821
822
852
853
875
878
879
880
881
938
967
973
1009
1037
1062
1063
1089
1114
1115
1116
1124
1131
1145
1146
1147
1148
1152
1153
1154
1155
1158
1160
1169
1171
1177
1196
1197
1198
1199
1200
1204
1206
1237


In [42]:
blink = Blink(**df['blinking_data'].iloc[69])
blink

Blink(intensity=0.7057374238967895, asymmetry=0.027345991134643532, is_blinking=True, rz_score=2.6695881547741345, is_anomalous=True, continuos_anomaly=False, part_of_anomalous_range=None)

In [46]:
result = await agent.run(user_prompt=f"analyze the data,{blink}")
print(result.output)

**Data Analysis**

The provided data consists of several metrics related to object detection or objectness, likely used in computer vision or machine learning applications. Here's a breakdown of the information:

1. **Intensity**: 0.7057374238967895
	* This is a value between 0 and 1 representing the intensity aspect. A higher value typically indicates a stronger signal or feature detection.
2. **Asymmetry**: 0.027345991134643532
	* This measures the asymmetry of the detected object, with values close to 0 indicating symmetry and values close to 1 indicating significant asymmetry.
3. **is_blinking**: True
	* A boolean value indicating whether the current detection is related to a blinking event (e.g., in face detection or surveillance applications).
4. **rfz_score** (or `rz_score`): 2.6695881547741345
	* This appears to be an anomaly score, with higher values indicating greater uncertainty or detectability of the object.
5. **is_anomalous**: True
	* A boolean value confirming whether t

#### Let's try Groq

In [9]:
api_key = os.getenv("GROK_API_KEY")
model = GroqModel(
    'llama-3.3-70b-versatile', provider=GroqProvider(api_key=api_key)
)
agent = Agent(model)
result = await agent.run(user_prompt="hii! how are you?")
print(result.output)

Hello. I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to chat with you. How about you? How's your day going?
